In [1]:
#Upgrade dependencies
!pip install --upgrade pip
!pip install --upgrade scikit-learn

  Using cached pip-20.3.3-py2.py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.3
    Uninstalling pip-20.3:
      Successfully uninstalled pip-20.3
     |████████████████████████████████| 22.2 MB 12.2 MB/s eta 0:00:01     |█████████████████████▊          | 15.1 MB 12.2 MB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.1
    Uninstalling scikit-learn-0.22.1:
      Successfully uninstalled scikit-learn-0.22.1


In [2]:
import numpy as np                                
import matplotlib.pyplot as plt   
from IPython.display import Image                 
from IPython.display import display               
from sklearn.datasets import dump_svmlight_file   
from time import gmtime, strftime                 
import sys                                        
import math                                       
import json
import boto3
import sagemaker

bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/fifa19'
role = sagemaker.get_execution_role()

print('Bucket:\n{}'.format(bucket))

Bucket:
sagemaker-us-east-1-172389125218


In [6]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
  
df = pd.read_excel('default of credit card clients.xls')

print('The shape of the training dataset is:', df.shape)

The shape of the training dataset is: (30001, 25)


In [7]:
df.head(10)

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,female,university,married,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,female,university,single,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,female,university,single,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,female,university,married,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,5,50000,male,university,married,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
6,6,50000,male,graduate school,single,37,0,0,0,0,...,19394,19619,20024,2500,1815,657,1000,1000,800,0
7,7,500000,male,graduate school,single,29,0,0,0,0,...,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
8,8,100000,female,university,single,23,0,-1,-1,0,...,221,-159,567,380,601,0,581,1687,1542,0
9,9,140000,female,others,married,28,0,0,2,0,...,12211,11793,3719,3329,0,432,1000,1000,1000,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30001 entries, 0 to 30000
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  30001 non-null  object
 1   X1          30001 non-null  object
 2   X2          30001 non-null  object
 3   X3          29670 non-null  object
 4   X4          29678 non-null  object
 5   X5          30001 non-null  object
 6   X6          30001 non-null  object
 7   X7          30001 non-null  object
 8   X8          30001 non-null  object
 9   X9          30001 non-null  object
 10  X10         30001 non-null  object
 11  X11         30001 non-null  object
 12  X12         30001 non-null  object
 13  X13         30001 non-null  object
 14  X14         30001 non-null  object
 15  X15         30001 non-null  object
 16  X16         30001 non-null  object
 17  X17         30001 non-null  object
 18  X18         30001 non-null  object
 19  X19         30001 non-null  object
 20  X20   

In [9]:
df.describe()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
count,30001,30001,30001,29670,29678,30001,30001,30001,30001,30001,...,30001,30001,30001,30001,30001,30001,30001,30001,30001,30001
unique,30001,82,3,6,4,64,12,12,12,12,...,21549,21011,20605,7944,7900,7519,6938,6898,6940,3
top,27966,50000,female,university,single,29,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
freq,1,3365,18112,14030,15964,1605,14737,15730,15764,16455,...,3195,3506,4020,5249,5396,5968,6408,6703,7173,23364


In [11]:
df.nunique()

Unnamed: 0    30001
X1               82
X2                3
X3                6
X4                4
X5               64
X6               12
X7               12
X8               12
X9               12
X10              11
X11              11
X12           22724
X13           22347
X14           22027
X15           21549
X16           21011
X17           20605
X18            7944
X19            7900
X20            7519
X21            6938
X22            6898
X23            6940
Y                 3
dtype: int64

In [14]:
df.columns

Index(['Unnamed: 0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9',
       'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19',
       'X20', 'X21', 'X22', 'X23', 'Y'],
      dtype='object')

In [15]:
numeric_features = ['Unnamed: 0', 'X1', 'X5', 'X6', 'X7', 'X8', 'X9', 
                   'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 
                   'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 
                   'X23']

text_categorical_features =  ['X2', 'X3', 'X4']

label_feature = ['Y']

In [18]:
df.drop_duplicates(inplace=True)

In [19]:
all_features = numeric_features + text_categorical_features + label_feature

In [20]:
label = 'Y'

In [21]:
df[all_features].head(10)

,Unnamed: 0,X1,X5,X6,X7,X8,X9,X10,X11,X12,...,X18,X19,X20,X21,X22,X23,X2,X3,X4,Y
0,ID,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SEX,EDUCATION,MARRIAGE,default payment next month
1,1,20000,24,2,2,-1,-1,-2,-2,3913,...,0,689,0,0,0,0,female,university,married,1
2,2,120000,26,-1,2,0,0,0,2,2682,...,0,1000,1000,1000,0,2000,female,university,single,1
3,3,90000,34,0,0,0,0,0,0,29239,...,1518,1500,1000,1000,1000,5000,female,university,single,0
4,4,50000,37,0,0,0,0,0,0,46990,...,2000,2019,1200,1100,1069,1000,female,university,married,0
5,5,50000,57,-1,0,-1,0,0,0,8617,...,2000,36681,10000,9000,689,679,male,university,married,0
6,6,50000,37,0,0,0,0,0,0,64400,...,2500,1815,657,1000,1000,800,male,graduate school,single,0
7,7,500000,29,0,0,0,0,0,0,367965,...,55000,40000,38000,20239,13750,13770,male,graduate school,single,0
8,8,100000,23,0,-1,-1,0,0,-1,11876,...,380,601,0,581,1687,1542,female,university,single,0
9,9,140000,28,0,0,2,0,0,0,11285,...,3329,0,432,1000,1000,1000,female,others,married,0


In [34]:
df = df[pd.notnull(df[label_feature])]

In [35]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df[all_features], test_size=0.2, shuffle=True, random_state=23)
val_data, test_data = train_test_split(test_data, test_size=0.2, shuffle=True, random_state=23)